## Deep Research Agent Development Notebook

### Tools used: OpenAI SDK, Python, Sendgrid, uv Package Manager

Agent system for generating deep research reports using OpenAI SDK

In [1]:
# imports

from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

### Testing Web Search Tool

### Search Agent

In [3]:
# system prompt taken from OpenAI's documentation for web search tool

INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    # using hosted tool from openai
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    # set to required to ensure the agent uses the tool
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**LangChain** is a modular framework that enables developers to build applications powered by large language models (LLMs). It supports integration with various LLMs, provides tools for prompt engineering, and facilitates the creation of complex workflows by chaining together different models and data sources. This flexibility makes it suitable for tasks ranging from conversational agents to document analysis. ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))

**AutoGen**, developed by Microsoft, specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. It offers features like multi-agent orchestration and runtime convergence for scalable applications, making it ideal for enterprise-level applications needing event-driven workflows, such as customer service automation. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**CrewAI** adopts a role-based agent collaboration approach, facilitating the creation of specialized agents that collaborate on complex projects akin to a team environment. It features dynamic task planning, real-time performance monitoring, and orchestration of a variety of agents as distinct workers, making it suitable for complex projects requiring teamwork among agents, such as software development or project management. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**LangGraph**, built on top of LangChain, focuses on managing stateful, branching processes using a graph-based architecture for agent interaction. It provides advanced error handling, support for complex stateful interactions, and multi-agent workflows, making it ideal for applications requiring extensive data retrieval and knowledge fusion, especially in research settings. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**Eliza** is an open-source, Web3-friendly AI agent operating system that integrates seamlessly with Web3 applications. It allows for reading and writing blockchain data, interacting with smart contracts, and is fully controlled by the user through regular TypeScript programs. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

**Agent Lightning** is a flexible and extensible framework that enables reinforcement learning-based training of LLMs for any AI agent. It achieves complete decoupling between agent execution and training, allowing seamless integration with existing agents developed via diverse ways, with almost zero code modifications. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

**AutoAgent** is a fully-automated and zero-code framework for LLM agents, enabling users to create and deploy LLM agents through natural language alone. It comprises components like Agentic System Utilities, LLM-powered Actionable Engine, Self-Managing File System, and Self-Play Agent Customization module, facilitating efficient and dynamic creation and modification of tools, agents, and workflows without coding requirements or manual intervention. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks represent the forefront of AI agent development in 2025, each catering to different needs and applications in the rapidly evolving field of artificial intelligence. 

### Look at the trace for the search agent

https://platform.openai.com/traces

### Use structured output to specify the format of the response 

### Planner Agent

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    # we add reason for reasoning which is like Chain of Thought prompting
    # add before query to encourage CoT
    # each field description is used by the model to understand the output
    # generating reason first will have the next token prediction of the query to be more aligned with the reason
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")

# we can nest pydantic objects. This is useful for complex queries.
# one defines the schema of the output 
# another one create a list of the objects of the first one
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

# Review: When we ask agent to output a pydantic object, it is not magically converted to a pydantic object.
# It actually use the JSON schema from the pydantic object to generate the output.

In [6]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most current AI agent frameworks released or updated in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore various categories and types of AI frameworks that are trending in 2025.', query='AI frameworks comparison 2025'), WebSearchItem(reason='To gather information about notable advancements or updates in AI agent technologies for the year 2025.', query='AI agent technologies advancements 2025')]


### Email Agent




In [ ]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("dnelfhmi@gmail.com") 
    to_email = To("dnelfhmi@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11fc7dbc0>, strict_json_schema=True, is_enabled=True)

In [10]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



### Writer Agent

In [9]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### Planning and Executing the Research (workflow)